In [ ]:
import sqlalchemy as sa
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import pandas as pd

In [3]:
SERVER = 'localhost'
DATABASE = 'AdventureWorks2022'
 
connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;Encrypt=no'
 
connection_url = URL.create('mssql+pyodbc', query={'odbc_connect': connection_string})
 
engine = create_engine(connection_url)